# **Setup**

In [1]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# Logging
from ngram_prep.ngram_acquire.logger import setup_logger

# Ngram pivot functions
from pathlib import Path
from ngram_prep.ngram_pivot.config import PipelineConfig
from ngram_prep.ngram_pivot.pipeline import run_pivot_pipeline

# Utilities
from ngram_prep.utilities.peek import *

## Log to file

In [2]:
setup_logger(
    db_path="/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db/ngram_download_20251027_071306.log')

# **Pivot Multigrams to Create Yearly Indices**

In [ ]:
pipeline_cfg = PipelineConfig(
    src_db=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"),
    dst_db=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db"),
    tmp_dir=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivot_tmp"),
    num_workers=30,
    num_initial_work_units=40,
    max_split_depth=100,
    work_unit_claim_order="random",
    split_check_interval_s=15.0,
    progress_every_s=600.0,
    mode="restart",
    max_items_per_bucket=50_000_000,
    max_bytes_per_bucket=5 * 1204 * 1024 * 1024,
    num_ingest_readers=3,
    ingest_buffer_shards=1
)

run_pivot_pipeline(pipeline_cfg)


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-10-26 16:38:56
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db
Temp directory:       /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivot_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              30
Initial work units:   40
Max split depth:      100
Split check interval: 5.0s

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       writ

# **Inspect the Procesed Database**
## `db_head`: Print the first _N_ key–value pairs

In [3]:
db_path = "/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db"

db_head(db_path, n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [1470] <UNK> <UNK> <UNK> <UNK> convenient
     Value: 1 occurrences in 1 documents

[ 2] Key:   [1470] <UNK> <UNK> <UNK> <UNK> eng
     Value: 1 occurrences in 1 documents

[ 3] Key:   [1470] <UNK> <UNK> <UNK> atomic energy
     Value: 1 occurrences in 1 documents

[ 4] Key:   [1470] <UNK> <UNK> <UNK> convenient one
     Value: 1 occurrences in 1 documents

[ 5] Key:   [1470] <UNK> <UNK> <UNK> eng <UNK>
     Value: 1 occurrences in 1 documents



## `db_peek`: Print _N_ key-value pairs starting at the specified key

In [4]:
db_path = "/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db"

db_peek(db_path, start_key="[2002] world trade center attack", n=5)


5 key-value pairs starting from 000007d2776f726c642074726164652063656e7465722061747461636b:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2002] world trade center attack <UNK>
     Value: 3,027 occurrences in 2,537 documents

[ 2] Key:   [2002] world trade center attack claim
     Value: 46 occurrences in 16 documents

[ 3] Key:   [2002] world trade center attack occur
     Value: 6 occurrences in 6 documents

[ 4] Key:   [2002] world trade center attack see
     Value: 1 occurrences in 1 documents

[ 5] Key:   [2002] world trade center attack victim
     Value: 13 occurrences in 13 documents



## `db_peek_prefix`: Print key-value pairs containing the specified prefix

In [26]:
db_path = "/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db"

db_peek_prefix(db_path, prefix="[2017] <UNK> risk <UNK> nuclear war", n=10)

10 key-value pairs with prefix 000007e13c554e4b3e207269736b203c554e4b3e206e75636c65617220776172:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2017] <UNK> risk <UNK> nuclear war
     Value: 279 occurrences in 220 documents

[ 2] Key:   [2017] <UNK> risk <UNK> nuclear warfare
     Value: 7 occurrences in 7 documents

